In [1]:
import torch as t
from torch import nn
from torch.utils.data import Dataset, DataLoader
import plotly.express as px
from IPython.display import display
import pandas as pd
import numpy as np
import copy
from fancy_einsum import einsum
from dataclasses import dataclass
from tqdm.notebook import tqdm_notebook

from einops import rearrange, reduce, repeat

import utils
import cnn_modules as cm
import transformer_modules as tm

In [2]:
def single_head_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of self-attention (see the "Self-Attention in Detail" section of the Illustrated Transformer).

    With this function, you can ignore masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [3]:
def single_head_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of masked self-attention.

    See "The Decoder Side" section of the Illustrated Transformer for an explanation of masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    batch_size, seq_len, embed_size = Q.shape
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    
    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values
    scores = scores.masked_fill(mask==0, -1e9)
    
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [4]:
def multihead_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
    '''
    Implements multihead masked attention on the matrices Q, K and V.

    Q: shape (batch, seq, nheads*headsize)
    K: shape (batch, seq, nheads*headsize)
    V: shape (batch, seq, nheads*headsize)

    returns: shape (batch, seq, nheads*headsize)
    '''
    Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

    batch_size, seq_len, nheads, headsize = Q.shape
    scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
    scores /= Q.shape[-1] ** 0.5

    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values where mask==0
    scores = scores.masked_fill(mask==0, -1e9)

    scores = t.softmax(scores, dim=-1)
    Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
    Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
    return Z

In [5]:
T = t.randn(2, 10, 4)

In [6]:
multihead_masked_attention(T, T, T, 2)

tensor([[[ 1.0647, -0.8555,  0.2480, -0.1498],
         [ 1.4286, -2.0063,  0.2520, -1.0636],
         [ 1.7776,  0.2491, -0.2070, -2.1021],
         [-1.5945, -0.6252, -0.5244,  0.1213],
         [ 0.0352,  0.2913,  0.0253,  0.1864],
         [-1.6392,  0.0229,  1.1604,  1.5212],
         [ 1.7266,  0.1053,  0.9466, -0.0786],
         [ 2.8077,  0.8220,  0.2012,  1.4214],
         [ 1.5614, -0.4841,  1.2195,  1.1147],
         [-0.5401,  1.3954,  1.5943, -0.9236]],

        [[-0.2820,  0.8137, -0.8170,  0.8693],
         [-2.0363, -0.8972,  0.8406, -1.2989],
         [ 0.3519,  0.8452, -0.8965,  0.6586],
         [-0.7222,  3.5021, -0.6157,  0.2740],
         [-0.0628, -0.1239, -0.2267,  0.4276],
         [-0.3916, -1.4194, -0.7385, -0.0101],
         [ 0.1012, -1.5824,  0.3545, -0.7094],
         [ 2.6165,  1.2206, -1.0841,  0.9312],
         [ 1.6377,  0.1355,  1.3437,  1.8582],
         [ 0.7903, -0.2151,  0.7791,  0.5341]]])

In [7]:
class MultiheadMaskedAttention(nn.Module):
    W_QKV: nn.Linear
    W_O: nn.Linear

    def __init__(self, hidden_size: int, num_heads: int):
        super().__init__()
        self.num_heads = num_heads
        self.query_size = int(hidden_size / num_heads)
        self.qkv = cm.Linear(hidden_size, 3*hidden_size)
        self.ff = cm.Linear(hidden_size, hidden_size)

    def multihead_masked_attention(self, Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
        '''
        Implements multihead masked attention on the matrices Q, K and V.

        Q: shape (batch, seq, nheads*headsize)
        K: shape (batch, seq, nheads*headsize)
        V: shape (batch, seq, nheads*headsize)

        returns: shape (batch, seq, nheads*headsize)
        '''
        Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

        batch_size, seq_len, nheads, headsize = Q.shape
        scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
        scores /= Q.shape[-1] ** 0.5

        # create lower-left triangle of ones, including the diagonal
        mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
        # fill with close-to-neg-inf values where mask==0
        scores = scores.masked_fill(mask==0, -1e9)

        scores = t.softmax(scores, dim=-1)
        Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
        Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
        return Z

    def forward(self, x: t.Tensor) -> t.Tensor:
        '''
        x: shape (batch, seq, hidden_size)

        Return: shape (batch, seq, hidden_size)
        '''
        out = self.qkv(x)
        Q, K, V = t.tensor_split(out, 3, dim=-1)

        Z = self.multihead_masked_attention(Q, K, V, self.num_heads)

        return self.ff(Z)

In [8]:
@dataclass(frozen=True)
class TransformerConfig:
    '''Constants used throughout your decoder-only transformer model.'''

    num_layers: int
    num_heads: int
    vocab_size: int
    hidden_size: int
    max_seq_len: int
    dropout: float = 0.1
    layer_norm_epsilon: float = 1e-05

In [9]:
config = TransformerConfig(
    num_layers=4, 
    num_heads=2, 
    vocab_size=500, 
    hidden_size=64,
    max_seq_len=100,
    dropout=0.1)

In [10]:
class MLP(nn.Module):

    def __init__(self, hidden_size, dropout):
        super().__init__()
        self.linear1 = cm.Linear(hidden_size, 4 * hidden_size)
        self.gelu = tm.GELU()
        self.linear2 = cm.Linear(4 * hidden_size, hidden_size)
        self.dropout = tm.Dropout(p=dropout)

    def forward(self, x):
        out = self.gelu(self.linear1(x))
        out = self.dropout(self.linear2(out))
        return out

In [11]:
class DecoderBlock(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.attn = tm.MultiheadMaskedAttention(config.hidden_size, config.num_heads)
        self.lnorm1 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)
        self.mlp = tm.MLP(config.hidden_size, config.dropout)
        self.lnorm2 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)

    def forward(self, x: t.Tensor) -> t.Tensor:
        normed_attn = self.lnorm1(self.attn(x))
        out = normed_attn + x
        normed_mlp = self.lnorm2(self.mlp(out))
        out = normed_mlp + out
        return out


In [12]:
class DecoderOnlyTransformer(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.emb = nn.Embedding(config.vocab_size, config.hidden_size)
        self.pos_enc = tm.PositionalEncoding(config.max_seq_len, config.hidden_size)
        self.dropout = tm.Dropout(p=config.dropout)

        decoders = [DecoderBlock(config) for l in range(config.num_layers)]
        self.decoders = nn.Sequential(*decoders)
        
        self.post_norm = tm.LayerNorm(config.hidden_size)

    def forward(self, x: t.Tensor) -> t.Tensor:
        embedding = self.emb(x.long())
        embedding = self.pos_enc(embedding)
        embedding = self.dropout(embedding)
        embedding = embedding.to(t.float32)

        out = self.decoders(embedding)
        out = self.post_norm(out)

        out = einsum("B S E, V E -> B S V", out, self.emb.weight)

        return out

## Testing Transformer

In [13]:
from torch.utils.data import Dataset

class ReverseNumberDataset(Dataset):
    def __init__(self, seq_len, total_size):
        self.seq_len = seq_len
        self.total_size = total_size

    def __len__(self):
        return self.total_size

    def __getitem__(self, idx):
        text = t.randint(0, 9, (self.seq_len, ))
        label = text.flip(dims=[0])
        sample = (text, label)
        return sample

In [14]:
num_ds = ReverseNumberDataset(6, 1_000_000)
num_ds[5]

(tensor([5, 5, 0, 4, 1, 3]), tensor([3, 1, 4, 0, 5, 5]))

In [15]:
trainloader = DataLoader(num_ds, batch_size=256, shuffle=True)

In [16]:
config = TransformerConfig(
    num_layers=2, 
    num_heads=4, 
    vocab_size=10, 
    hidden_size=128,
    max_seq_len=6,
    dropout=0.1)

In [17]:
from typing import Callable


epochs = 1
loss_fn = nn.CrossEntropyLoss()
batch_size = 256

MODEL_FILENAME = "./w1d2_transformer_digits.pt"
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")

def train_transformer(trainloader: DataLoader, epochs: int, loss_fn: Callable) -> list:
    '''
    Defines a Transformer from our custom modules, and trains it on the reversed digit dataset.
    '''

    model = DecoderOnlyTransformer(config).to(device).train()
    optimizer = t.optim.Adam(model.parameters())
    loss_list = []
    accuracy_list = []

    for epoch in range(epochs):

        progress_bar = tqdm_notebook(trainloader)
        for (x, y) in progress_bar:

            x = x.to(t.float32)
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            print(logits.shape)
            logits = rearrange(logits, 'B S V -> (B S) V')
            print(logits)
            print(y.shape)
            y = rearrange(y, 'B S -> (B S)')


            loss = loss_fn(logits, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_list.append(loss.item())

            with t.inference_mode():
                model.eval()
                preds = model(t.tensor([[1,2,3,4,5,6]]).to(device))
                preds = preds.argmax(dim=-1)
                model.train()

            progress_bar.set_description(f"Epoch = {epoch}, Preds = {preds.squeeze()}, Loss = {loss.item():.4f}")

    print(f"Saving model to: {MODEL_FILENAME}")
    t.save(model, MODEL_FILENAME)
    return loss_list, accuracy_list

loss_list, accuracy_list = train_transformer(trainloader, epochs, loss_fn)

fig = px.line(y=loss_list, template="simple_white")
fig.update_layout(title="Cross entropy loss on number sequences", yaxis_range=[0, max(loss_list)])
fig.show()

  0%|          | 0/3907 [00:00<?, ?it/s]

torch.Size([256, 6, 10])
tensor([[ -6.9073,  -3.7603,  12.0222,  ...,  -1.6182,  47.5219,  -0.5380],
        [ 12.5366, -13.9293,  -5.8833,  ...,  31.1231, -13.7577,  -0.7697],
        [-11.1538,   5.3634,  18.8077,  ...,  17.8596,  -9.5417,  -1.5267],
        ...,
        [ 13.4708, -13.9995, -14.9334,  ...,  29.0654,  -4.0873,   0.4564],
        [-10.8156,  -1.4065,  26.1522,  ...,  10.3581,  -4.3370,   3.3703],
        [ -1.6986, -12.0968,  21.1649,  ...,   6.7432,  40.5860,   1.1177]],
       grad_fn=<ReshapeAliasBackward0>)
torch.Size([256, 6])
torch.Size([256, 6, 10])
tensor([[ 39.0922,  -1.0006,  13.8367,  ...,  10.1821,  -4.7251,  12.1346],
        [ 14.3882,  -4.4207,   4.9605,  ...,  28.7057,  -4.9647,   5.6452],
        [  6.1134,  -1.5780,   8.2076,  ...,  55.8461, -10.3192,  12.6168],
        ...,
        [ 11.5101,   3.6963,  -3.4259,  ...,  -0.2284,   7.3653,   1.0144],
        [  9.6767,  -1.2530,   0.3356,  ...,  23.8977,  -0.6481,   1.7224],
        [ -5.3048,   4.243

KeyboardInterrupt: 